In [2]:
import numpy as np
import pandas as pd
#---------------------csv location------------------------------------------------
dataLocation = r'/Users/yuliang/Desktop/opening_data/high/data_high.csv'
data_normal = pd.read_csv(dataLocation)
data_normal_used = data_normal[['oindex','x','y','match','team','player','won','tsync','tier','zone']]
#--------------------initial state-------------------------------------------
data_select1 = data_normal_used.replace(to_replace='bottomlane radiant', value='bottomlane')
data_select2 = data_select1.replace(to_replace='bottomlane dire', value='bottomlane')
data_select3 = data_select2.replace(to_replace='midlane radiant', value='midlane')
data_select4 = data_select3.replace(to_replace='midlane dire', value='midlane')
data_select5 = data_select4.replace(to_replace='toplane radiant', value='toplane')
data_select6 = data_select5.replace(to_replace='toplane dire', value='toplane')
data_select7 = data_select6.replace(to_replace='base1 radiant', value='base')
data_select8 = data_select7.replace(to_replace='base2 dire', value='base')
data_select9 = data_select8.replace(to_replace='void radiant', value='void')
data_select10 = data_select9.replace(to_replace='void dire', value='void')
data_select11 = data_select10.replace(to_replace='jungle radiant', value='jungle')
data_change = data_select11.replace(to_replace='jungle dire', value='jungle')
#print(data_change)
#--------------------initial state-------------------------------------------
data_initial_1 = data_change[data_change['tsync']==0]
#--------------------initial state lose---------------------------------------
data_initial_lose1 = data_initial_1[data_initial_1['won']==0]
data_initial_lose2 = data_initial_lose1.sort(['match'], ascending = [True]).reset_index()
data_initial_lose = data_initial_lose2[['index','x','y','oindex','match','team','player','won','tier','zone']]

data_initial_losedr = data_initial_lose.drop_duplicates('match')

def open_strategy (df, x, y, player_index, real_index, player_name, zone, first_zone, x_value, y_value):
    i = player_index #new player index, start from 0
    initial_state = df.get_value(real_index, zone)
    x_var = df.get_value(real_index, x)
    y_var = df.get_value(real_index, y)
    if ((initial_state != 'base') & (initial_state != 'void')):
        first_zone[i] = initial_state
        x_value[i] = x_var
        y_value[i] = y_var
    else:
        for index in range((real_index+1), len(df.index)):
            current_state = df.get_value(index, zone)
            if ((current_state != 'base') & (current_state != 'void')):
                first_zone[i] = current_state
                x_value[i] = df.get_value(index, x)
                y_value[i] = df.get_value(index, y)
                break
    return first_zone, x_value, y_value
            
array1 = ['a' for x in range(len(data_initial_lose.index))]
array2 = ['a' for x in range(len(data_initial_lose.index))]

array3 = ['a' for x in range(len(data_initial_lose.index))]
array4 = ['a' for x in range(len(data_initial_lose.index))]

array5 = ['a' for x in range(len(data_initial_lose.index))]
array6 = ['a' for x in range(len(data_initial_lose.index))]

for index in range(len(data_initial_lose.index)):
    real_index = data_initial_lose.get_value(index, 'index')
    array2, array4, array6 = open_strategy(data_change, 'x', 'y', index, real_index,'player','zone', array1, array3, array5)
#print(array2)
#print(array4)
#print(array6)

#len(array2)
match_strategy = [[0 for x in range (5)] for x in range(len(data_initial_losedr.index))]

for index in range(len(data_initial_losedr.index)):
    new_index = index*5
    new_index_stop = new_index + 5
    for i in range(new_index, new_index_stop):
        if array2[i] == 'toplane':
            match_strategy[index][0] = match_strategy[index][0] + 1
        elif array2[i] == 'midlane':
            match_strategy[index][1] = match_strategy[index][1] + 1
        elif array2[i] == 'bottomlane':
            match_strategy[index][2] = match_strategy[index][2] + 1
        elif array2[i] == 'jungle':
            match_strategy[index][3] = match_strategy[index][3] + 1
        else:
            match_strategy[index][4] = match_strategy[index][4] + 1
#print(match_strategy)
player_strategy = [[0 for x in range (5)] for x in range(len(array2))]
for index in range(len(data_initial_losedr.index)):
    new_index = index*5
    new_index_stop = new_index + 5
    for i in range(new_index, new_index_stop):
        player_strategy[i][0] = match_strategy[index][0]
        player_strategy[i][1] = match_strategy[index][1]
        player_strategy[i][2] = match_strategy[index][2]
        player_strategy[i][3] = match_strategy[index][3]
        player_strategy[i][4] = match_strategy[index][4]


#print(player_strategy)


new_str = ['a' for x in range(len(array2))]

for i in range(len(array2)):
    new_str[i] = "".join(str(x) for x in player_strategy[i])
#print(new_str)

strategy_series = pd.Series(new_str)

zone_series = pd.Series(array2)
x_series = pd.Series(array4)
y_series = pd.Series(array6)

df1 = pd.DataFrame(strategy_series, columns = ['match_strategy'])
df2 = pd.DataFrame(zone_series, columns = ['zone'])
df3 = pd.DataFrame(x_series, columns = ['x'])
df4 = pd.DataFrame(y_series, columns = ['y'])
data_final = data_initial_lose[['match','team','player','won','tier']]

df = pd.concat([data_final, df3, df4, df2, df1], join='outer', axis=1)
#print(df)
df.to_csv('/Users/yuliang/Desktop/high_open_lose.csv', index=False, sep=',' )          
print("succeed")


succeed
